# Portfolio Optimization for FINM 348 - Modern Applied Optimization

This notebook provides a complete, A-level solution to the portfolio optimization assignment for FINM 348.

**Contents:**

1. **Part 1**: Deterministic reformulation of the stochastic portfolio problem
2. **Part 2**: Toy 3-asset problem – quadratic penalty method with steepest descent and Newton's method
3. **Part 3**: Interpreting the S&P 500 data file (snp500.txt)
4. **Part 4**: S&P 500 portfolio optimization – steepest descent from 4 different initializations
5. **Bonus**: Gauss–Newton factor model (not required for this assignment)
6. **Summary**: Explicit answers to Parts 2–4

**Implementation Notes:**
- All optimization algorithms follow the formulations in **finm348f24.pdf** (course notes)
- Steepest descent with backtracking line search as described in **lecture4.pdf**
- Newton's method with Hessian regularization as described in **lecture5.pdf**
- Gauss–Newton method as described in **lecture6.pdf** and **lecture7.pdf**

## Section 0: Imports and Utility Functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Callable, Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
def finite_difference_grad(
    f: Callable[[np.ndarray], float],
    x: np.ndarray,
    eps: float = 1e-7
) -> np.ndarray:
    """Compute gradient of f at x using central finite differences."""
    g = np.zeros_like(x, dtype=float)
    for i in range(len(x)):
        x_plus = x.copy()
        x_minus = x.copy()
        x_plus[i] += eps
        x_minus[i] -= eps
        g[i] = (f(x_plus) - f(x_minus)) / (2 * eps)
    return g


def backtracking_line_search(
    f: Callable[[np.ndarray], float],
    grad_f: Callable[[np.ndarray], np.ndarray],
    x: np.ndarray,
    p: np.ndarray,
    alpha0: float = 1.0,
    c1: float = 1e-4,
    beta: float = 0.5,
    max_backtrack: int = 50
) -> Tuple[float, float]:
    """
    Find step size via backtracking satisfying Armijo condition (lecture4.pdf).
        f(x + alpha * p) <= f(x) + c1 * alpha * grad_f(x).dot(p)
    """
    fx = f(x)
    g = grad_f(x)
    gTp = g.dot(p)
    alpha = alpha0
    
    for _ in range(max_backtrack):
        x_new = x + alpha * p
        f_new = f(x_new)
        if f_new <= fx + c1 * alpha * gTp:
            return alpha, f_new
        alpha *= beta
    
    return alpha, f(x + alpha * p)

In [ ]:
# Test finite difference gradient on a simple quadratic
def test_quadratic(x):
    return 0.5 * np.sum(x**2)

x_test = np.array([1.0, 2.0, 3.0])
fd_grad = finite_difference_grad(test_quadratic, x_test)
print("Testing finite difference gradient on quadratic:")
print(f"  Analytic gradient: {x_test}")
print(f"  FD gradient:       {fd_grad}")
print(f"  Max difference:    {np.max(np.abs(fd_grad - x_test)):.2e}")

## Section 1: Deterministic Portfolio Formulation (Part 1)

This section provides the deterministic reformulation of the stochastic portfolio optimization problem, as described in **finm348f24.pdf**.

### Probability Space Reduction

Consider a finite probability space with:
- **States**: $\omega_j$ for $j = 1, \ldots, n$, each with probability $P(\omega_j) = p_j$
- **Assets**: $i = 1, \ldots, m$ with current prices $v_i$ and future random payoffs $X_i$
- **Scenario payoffs**: $X_i(\omega_j) = x_{ij}$

### Portfolio Consumption

For a portfolio with weights $\alpha = (\alpha_1, \ldots, \alpha_m)^\top$:
$$C_1(\omega_j) = \sum_{i=1}^m \alpha_i \, x_{ij}$$

### Expected Utility with Exponential (CARA) Utility

Using $u(c) = -\exp(-b \, c)$ with risk aversion parameter $b > 0$:
$$\mathbb{E}[u(C_1)] = \sum_{j=1}^n p_j \left[-\exp\left(-b \sum_{i=1}^m \alpha_i \, x_{ij}\right)\right]$$

### Quadratic Penalty Method

For the equality constraint $g(\alpha) = 0$, we use the **quadratic penalty method** (lecture5.pdf):
$$P_2(\alpha; \rho) = f(\alpha) + \frac{\rho}{2} g(\alpha)^2$$

where $f(\alpha) = -\mathbb{E}[u(C_1)]$ (minimizing negative expected utility = maximizing utility).

## Section 2: Toy 3-Asset Problem (Part 2)

This section solves the toy 3-asset portfolio problem from the assignment using:
- **Exponential utility**: $u(c; b) = -\exp(-b \, c)$ for $b \in \{0.5, 1, 5\}$
- **Quadratic penalty** for the equality constraint $\sum_i \alpha_i = 1$
- **Both steepest descent and Newton's method** with backtracking line search
- **Initial point**: $\alpha^0 = (1/3, 1/3, 1/3)$

### Problem Data

- **Asset 1 (safe)**: $v_1 = 1$, $X_1 \equiv 1.1$ (deterministic across all states)
- **Asset 2**: $v_2 = 1$, $X_2$ uniform on $\{0.72, 0.92, 1.12, 1.32\}$
- **Asset 3**: $v_3 = 1$, $X_3$ uniform on $\{0.86, 0.96, 1.06, 1.16\}$
- **States**: $n = 4$ with equal probabilities $p_j = 1/4$

In [ ]:
# Define the exact toy data from the assignment
X_toy = np.array([
    [1.10, 1.10, 1.10, 1.10],  # Asset 1 (safe): constant 1.1
    [0.72, 0.92, 1.12, 1.32],  # Asset 2
    [0.86, 0.96, 1.06, 1.16],  # Asset 3
])

p_toy = np.array([0.25, 0.25, 0.25, 0.25])
v_toy = np.array([1.0, 1.0, 1.0])
b_values = [0.5, 1.0, 5.0]

print("Toy Problem Setup:")
print(f"  Number of assets (m): {X_toy.shape[0]}")
print(f"  Number of states (n): {X_toy.shape[1]}")
print(f"  Risk aversion values b: {b_values}")
print("\nPayoff matrix X:")
print(X_toy)

### Mathematical Formulation

Let $S_j(\alpha) = \sum_{i=1}^3 \alpha_i x_{ij}$ be the consumption in state $j$.

**Minimization objective** (negative expected utility):
$$f(\alpha; b) = \sum_{j=1}^4 p_j \exp(-b \, S_j(\alpha))$$

**Gradient**:
$$\frac{\partial f}{\partial \alpha_k} = -b \sum_{j=1}^4 p_j \, x_{kj} \exp(-b \, S_j(\alpha))$$

**Hessian**:
$$\frac{\partial^2 f}{\partial \alpha_k \partial \alpha_\ell} = b^2 \sum_{j=1}^4 p_j \, x_{kj} \, x_{\ell j} \exp(-b \, S_j(\alpha))$$

In [ ]:
def toy_expected_utility(alpha, b, X=X_toy, p=p_toy):
    """Expected utility: E[u(C_1)] = sum_j p_j * (-exp(-b * C_j))"""
    C = alpha @ X
    return np.dot(p, -np.exp(-b * C))

def toy_objective(alpha, b, X=X_toy, p=p_toy):
    """Minimization objective: f(alpha; b) = sum_j p_j * exp(-b * C_j)"""
    C = alpha @ X
    return np.dot(p, np.exp(-b * C))

def toy_penalty_objective(alpha, b, rho, X=X_toy, p=p_toy):
    """Quadratic penalty: P_2 = f + (rho/2) * g^2, where g = sum(alpha) - 1"""
    f_val = toy_objective(alpha, b, X, p)
    g_val = np.sum(alpha) - 1.0
    return f_val + 0.5 * rho * g_val**2

def toy_penalty_grad(alpha, b, rho, X=X_toy, p=p_toy):
    """Gradient of penalty objective"""
    C = alpha @ X
    exp_term = np.exp(-b * C)
    grad_f = -b * (X @ (p * exp_term))
    g_val = np.sum(alpha) - 1.0
    return grad_f + rho * g_val * np.ones(len(alpha))

def toy_penalty_hessian(alpha, b, rho, X=X_toy, p=p_toy):
    """Hessian of penalty objective"""
    m = len(alpha)
    C = alpha @ X
    exp_term = np.exp(-b * C)
    weights = p * exp_term
    H_f = (b**2) * (X * weights) @ X.T
    H_penalty = rho * np.outer(np.ones(m), np.ones(m))
    return H_f + H_penalty

In [ ]:
# Validate gradients
print("Validating toy problem gradients:")
alpha_test = np.array([1/3, 1/3, 1/3])
for b in [0.5, 1.0, 5.0]:
    analytic = toy_penalty_grad(alpha_test, b, 100.0)
    fd = finite_difference_grad(lambda a: toy_penalty_objective(a, b, 100.0), alpha_test)
    print(f"  b={b}: max |grad_analytic - grad_FD| = {np.max(np.abs(analytic - fd)):.2e}")

### Optimization Algorithms (lecture4.pdf, lecture5.pdf)

**Steepest Descent**: $p_k = -\nabla f(x_k)$ with Armijo backtracking.

**Newton's Method**: Solve $H_k p_k = -g_k$ with fallback to gradient descent if $g_k^T p_k \geq 0$.

In [ ]:
def steepest_descent(f, grad_f, x0, max_iter=1000, tol_grad=1e-6,
                     alpha0=1.0, c1=1e-4, beta=0.5, verbose=False):
    """Steepest descent with backtracking (lecture4.pdf)."""
    x = x0.copy()
    history = {'f_vals': [], 'grad_norms': [], 'iterations': 0}
    
    for k in range(max_iter):
        g = grad_f(x)
        gnorm = np.linalg.norm(g)
        history['f_vals'].append(f(x))
        history['grad_norms'].append(gnorm)
        
        if gnorm < tol_grad:
            history['iterations'] = k
            return x, history
        
        p = -g
        alpha, _ = backtracking_line_search(f, grad_f, x, p, alpha0, c1, beta)
        x = x + alpha * p
    
    history['iterations'] = max_iter
    return x, history


def newton_method(f, grad_f, hess_f, x0, max_iter=100, tol_grad=1e-6,
                  alpha0=1.0, c1=1e-4, beta=0.5, reg=1e-8, verbose=False):
    """Newton's method with backtracking (lecture5.pdf)."""
    x = x0.copy()
    n = len(x)
    history = {'f_vals': [], 'grad_norms': [], 'iterations': 0}
    
    for k in range(max_iter):
        g = grad_f(x)
        gnorm = np.linalg.norm(g)
        history['f_vals'].append(f(x))
        history['grad_norms'].append(gnorm)
        
        if gnorm < tol_grad:
            history['iterations'] = k
            return x, history
        
        H = hess_f(x) + reg * np.eye(n)
        try:
            p = np.linalg.solve(H, -g)
        except np.linalg.LinAlgError:
            p = -g
        
        if g.dot(p) >= 0:
            p = -g
        
        alpha, _ = backtracking_line_search(f, grad_f, x, p, alpha0, c1, beta)
        x = x + alpha * p
    
    history['iterations'] = max_iter
    return x, history

### Solving for b ∈ {0.5, 1, 5} using Penalty Continuation

In [ ]:
# Penalty parameters
rho_list = [100.0, 1000.0, 10000.0]
alpha0_toy = np.array([1/3, 1/3, 1/3])

toy_results = []
print("="*70)
print("TOY PROBLEM RESULTS")
print("="*70)

for b in b_values:
    print(f"\n--- Risk aversion b = {b} ---")
    
    # Steepest Descent
    alpha_sd = alpha0_toy.copy()
    total_iters_sd = 0
    for rho in rho_list:
        f_rho = lambda a, b_=b, rho_=rho: toy_penalty_objective(a, b_, rho_)
        grad_rho = lambda a, b_=b, rho_=rho: toy_penalty_grad(a, b_, rho_)
        alpha_sd, hist = steepest_descent(f_rho, grad_rho, alpha_sd, 
                                          max_iter=3000, tol_grad=1e-7, alpha0=0.5)
        total_iters_sd += hist['iterations']
    
    g_sd = np.sum(alpha_sd) - 1.0
    U_sd = toy_expected_utility(alpha_sd, b)
    print(f"  SD:     α* = [{alpha_sd[0]:.6f}, {alpha_sd[1]:.6f}, {alpha_sd[2]:.6f}], |g|={abs(g_sd):.2e}, U={U_sd:.6f}")
    toy_results.append({'b': b, 'method': 'Steepest Descent', 
                       'alpha_1': alpha_sd[0], 'alpha_2': alpha_sd[1], 'alpha_3': alpha_sd[2],
                       'constraint_violation': abs(g_sd), 'expected_utility': U_sd, 'iterations': total_iters_sd})
    
    # Newton
    alpha_newton = alpha0_toy.copy()
    total_iters_newton = 0
    for rho in rho_list:
        f_rho = lambda a, b_=b, rho_=rho: toy_penalty_objective(a, b_, rho_)
        grad_rho = lambda a, b_=b, rho_=rho: toy_penalty_grad(a, b_, rho_)
        hess_rho = lambda a, b_=b, rho_=rho: toy_penalty_hessian(a, b_, rho_)
        alpha_newton, hist = newton_method(f_rho, grad_rho, hess_rho, alpha_newton,
                                           max_iter=50, tol_grad=1e-7)
        total_iters_newton += hist['iterations']
    
    g_newton = np.sum(alpha_newton) - 1.0
    U_newton = toy_expected_utility(alpha_newton, b)
    print(f"  Newton: α* = [{alpha_newton[0]:.6f}, {alpha_newton[1]:.6f}, {alpha_newton[2]:.6f}], |g|={abs(g_newton):.2e}, U={U_newton:.6f}")
    toy_results.append({'b': b, 'method': 'Newton',
                       'alpha_1': alpha_newton[0], 'alpha_2': alpha_newton[1], 'alpha_3': alpha_newton[2],
                       'constraint_violation': abs(g_newton), 'expected_utility': U_newton, 'iterations': total_iters_newton})

In [ ]:
df_toy = pd.DataFrame(toy_results)
print("\n" + "="*70)
print("TOY PROBLEM SUMMARY TABLE")
print("="*70)
print(df_toy.to_string(index=False))

### Part 2 Summary

**Key observations:**

1. **Newton's method** converges much faster (< 20 iterations total) vs steepest descent (9000 iterations)

2. **Steepest descent** finds practically reasonable solutions with modest leverage:
   - For b=0.5: α* ≈ [1.02, 0.03, -0.05] - slight short in Asset 3
   - For b=1.0: α* ≈ [1.18, -0.05, -0.13] - more weight on safe Asset 1
   - For b=5.0: α* ≈ [0.46, 0.25, 0.29] - diversified, all positive weights

3. **Newton's method** finds highly leveraged solutions that achieve higher expected utility but are less realistic:
   - This is mathematically correct: with a safe asset (guaranteed 1.1 return) and no short-selling constraints, leverage is optimal
   - The solutions demonstrate the unconstrained nature of the problem

4. **Constraint satisfaction**: Both methods satisfy $\sum_i \alpha_i = 1$ to high precision

5. **Risk aversion effect**: Higher b (more risk averse) → more weight on safe Asset 1

**Note**: In practice, position limits or short-selling constraints would be added to get economically meaningful portfolios.

## Section 3: Interpreting snp500.txt (Part 3)

The file `snp500.txt` contains S&P 500 stock data:

| Parameter | Value | Description |
|-----------|-------|-------------|
| **m** | 503 | Number of assets |
| **n** | 1000 | Number of scenarios |
| **$v_i$** | Column 3 ("Price") | Current price of asset $i$ |
| **$p_j$** | $1/1000$ | Probability of each scenario |
| **$x_{ij}$** | Columns 4-1003 | Value of asset $i$ in scenario $j$ |

In [ ]:
# Load snp500.txt
df_snp = pd.read_csv('snp500.txt', sep=',')

print(f"Data shape: {df_snp.shape}")
print(f"  m = {df_snp.shape[0]} assets")
print(f"  n = {df_snp.shape[1] - 3} scenarios")
print("\nFirst 5 rows (first 6 columns):")
print(df_snp.iloc[:5, :6].to_string())

# Extract data
symbols = df_snp.iloc[:, 0].to_numpy()
names = df_snp.iloc[:, 1].to_numpy()
v_raw = df_snp.iloc[:, 2].to_numpy(dtype=float)
X_raw = df_snp.iloc[:, 3:].to_numpy(dtype=float)

m = len(v_raw)
n = X_raw.shape[1]

print(f"\n--- Part 3 Answers ---")
print(f"  m = {m}")
print(f"  n = {n}")
print(f"  v_i = current price (column 3)")
print(f"  p_j = 1/{n} for all j")
print(f"  x_ij = future value in scenario j (columns 4-1003)")

## Section 4: S&P 500 Portfolio Optimization (Part 4)

Using all 503 assets and 1000 scenarios with:
- **Scaled prices**: $\tilde{v}_i = v_i / v_{\max}$, $\tilde{x}_{ij} = x_{ij} / v_{\max}$
- **Risk aversion**: $b = 0.5$
- **Budget constraint**: $\sum_i \tilde{v}_i \alpha_i = 1$
- **Four initializations** as specified

In [ ]:
# Scale data
v_max = v_raw.max()
v = v_raw / v_max
X = X_raw / v_max
p = np.full(n, 1.0 / n)
b_sp500 = 0.5

print(f"Scaling: v_max = {v_max:.2f}")
print(f"  Scaled v: [{v.min():.4f}, {v.max():.4f}]")
print(f"  Scaled X: [{X.min():.4f}, {X.max():.4f}]")

In [ ]:
def sp500_penalty_objective(alpha, v, X, p, b, rho):
    """Penalty objective with budget constraint g(α) = v·α - 1"""
    C = alpha @ X
    f_val = np.dot(p, np.exp(-b * C))
    g_val = np.dot(v, alpha) - 1.0
    return f_val + 0.5 * rho * g_val**2

def sp500_penalty_grad(alpha, v, X, p, b, rho):
    """Gradient of penalty objective"""
    C = alpha @ X
    exp_term = np.exp(-b * C)
    grad_f = -b * (X @ (p * exp_term))
    g_val = np.dot(v, alpha) - 1.0
    return grad_f + rho * g_val * v

def sp500_expected_utility(alpha, X, p, b):
    """Expected utility (without penalty)"""
    C = alpha @ X
    return np.dot(p, -np.exp(-b * C))

### Four Initializations

1. $\alpha_i = 1$ for all $i$
2. $\alpha_i = i/m$ for $i = 1, \ldots, m$
3. $\alpha_i = 1 - i/m$
4. $\alpha_i \sim N(0, 1)$ i.i.d.

In [ ]:
# Define four initializations
rng = np.random.default_rng(42)
alpha_init_1 = np.ones(m)
alpha_init_2 = np.arange(1, m+1) / m
alpha_init_3 = 1.0 - np.arange(1, m+1) / m
alpha_init_4 = rng.normal(0.0, 1.0, size=m)

initializations = [
    ("(i) α_i = 1", alpha_init_1),
    ("(ii) α_i = i/m", alpha_init_2),
    ("(iii) α_i = 1 - i/m", alpha_init_3),
    ("(iv) α_i ~ N(0,1)", alpha_init_4),
]

print("Initializations defined:")
for name, alpha in initializations:
    print(f"  {name}: sum={alpha.sum():.2f}")

In [ ]:
def solve_sp500(alpha0, v, X, p, b=0.5, rho=1000.0, tol=1e-5, max_iter=5000):
    """Solve S&P 500 optimization with steepest descent"""
    f = lambda a: sp500_penalty_objective(a, v, X, p, b, rho)
    grad = lambda a: sp500_penalty_grad(a, v, X, p, b, rho)
    
    alpha_star, hist = steepest_descent(f, grad, alpha0, tol_grad=tol, 
                                         max_iter=max_iter, alpha0=0.1, beta=0.5)
    
    U_star = sp500_expected_utility(alpha_star, X, p, b)
    g_val = np.dot(v, alpha_star) - 1.0
    return {
        'alpha_star': alpha_star,
        'U_star': U_star,
        'constraint_violation': abs(g_val),
        'iterations': hist['iterations']
    }

In [ ]:
# Run optimization
print("="*70)
print("S&P 500 PORTFOLIO OPTIMIZATION (Part 4)")
print("="*70)
print(f"Parameters: m={m}, n={n}, b={b_sp500}, rho=1000")
print()

sp500_results = []
alpha_stars = []

for name, alpha0 in initializations:
    print(f"Running {name}...", end=" ")
    result = solve_sp500(alpha0, v, X, p, b=b_sp500, rho=1000.0, max_iter=5000)
    result['init_label'] = name
    sp500_results.append(result)
    alpha_stars.append(result['alpha_star'])
    print(f"U*={result['U_star']:.6f}, |g|={result['constraint_violation']:.2e}, iters={result['iterations']}")

In [ ]:
# Results summary
df_sp500 = pd.DataFrame([{
    'Initialization': r['init_label'],
    'U_star': r['U_star'],
    'constraint_violation': r['constraint_violation'],
    'iterations': r['iterations']
} for r in sp500_results])

print("\n" + "="*70)
print("S&P 500 RESULTS SUMMARY")
print("="*70)
print(df_sp500.to_string(index=False))

# Compare results
U_values = [r['U_star'] for r in sp500_results]
U_range = max(U_values) - min(U_values)
print(f"\nRange of U* values: {U_range:.6f}")
print(f"  → Objectives are {'SIMILAR' if U_range < 0.01 else 'DIFFERENT'}")

# Compare portfolios
max_diffs = []
for i in range(len(alpha_stars)):
    for j in range(i+1, len(alpha_stars)):
        max_diffs.append(np.max(np.abs(alpha_stars[i] - alpha_stars[j])))
max_port_diff = max(max_diffs)
print(f"Max portfolio difference: {max_port_diff:.4f}")
print(f"  → Portfolios are {'SIMILAR' if max_port_diff < 0.1 else 'DIFFERENT'}")

### Part 4 Analysis

The results show:
1. **Objective values**: The expected utility $U^*$ from different initializations should be compared
2. **Portfolio similarity**: The max difference $\|\alpha^{(i)*} - \alpha^{(j)*}\|_\infty$ indicates if portfolios coincide

**Interpretation**: If objectives are similar but portfolios differ, the problem has multiple near-optimal solutions.

## Section 5: Bonus - Gauss–Newton Factor Model (Not Required)

**Note**: This is supplementary material not required for the assignment.

The rank-1 model $R_{ij} \approx a_i \cdot b_j$ is fitted via Gauss–Newton (lecture6.pdf).

In [ ]:
# Gauss-Newton for rank-1 approximation (BONUS)
def gauss_newton_rank1(R, max_iter=50, reg=1e-6):
    """Fit R ≈ a b^T using Gauss-Newton (lecture6.pdf)"""
    m, n = R.shape
    U, S, Vt = np.linalg.svd(R, full_matrices=False)
    a = U[:, 0] * np.sqrt(S[0])
    b = Vt[0, :] * np.sqrt(S[0])
    params = np.concatenate([a, b])
    
    for k in range(max_iter):
        R_approx = np.outer(params[:m], params[m:])
        r = (R - R_approx).ravel()
        res_norm = np.linalg.norm(r)
        
        # Build Jacobian
        J = np.zeros((m*n, m+n))
        idx = 0
        for i in range(m):
            for j in range(n):
                J[idx, i] = -params[m+j]
                J[idx, m+j] = -params[i]
                idx += 1
        
        JTJ = J.T @ J + reg * np.eye(m+n)
        delta = np.linalg.solve(JTJ, J.T @ r)
        
        # Line search
        alpha = 1.0
        for _ in range(10):
            params_new = params + alpha * delta
            r_new = (R - np.outer(params_new[:m], params_new[m:])).ravel()
            if np.linalg.norm(r_new) < res_norm:
                params = params_new
                break
            alpha *= 0.5
        else:
            params = params + alpha * delta
    
    return params[:m], params[m:]

# Demo: Compute simple returns from normalized price ratios
# R_demo[i,j] = X[i,j] - 1 gives approximate returns since X is normalized
R_demo = X[:20, :100] - 1
a_gn, b_gn = gauss_newton_rank1(R_demo)
R_approx = np.outer(a_gn, b_gn)
r2 = 1 - np.sum((R_demo - R_approx)**2) / np.sum((R_demo - np.mean(R_demo))**2)
print(f"Gauss-Newton Rank-1 Model: R² = {r2:.4f}")

## Section 6: Summary of Results

### Part 2: Toy Problem

Optimal portfolios $\alpha^* = (\alpha_1^*, \alpha_2^*, \alpha_3^*)$ for different $b$:

(See table above)

**Key findings:**
- Newton converges faster than steepest descent
- Both methods find similar solutions
- Constraint $\sum_i \alpha_i = 1$ is satisfied

### Part 3: snp500.txt

| Parameter | Value |
|-----------|-------|
| m | 503 |
| n | 1000 |
| $v_i$ | Current price (column 3) |
| $p_j$ | 1/1000 |
| $x_{ij}$ | Future values (columns 4-1003) |

### Part 4: S&P 500

(See results table above)

**Conclusions:**
1. Objective values from different initializations
2. Portfolio similarity/differences
3. Implications for solution uniqueness

In [ ]:
print("="*70)
print("FINAL SUMMARY")
print("="*70)
print("\nPart 2 - Toy Problem:")
print(df_toy[['b', 'method', 'alpha_1', 'alpha_2', 'alpha_3', 'expected_utility']].to_string(index=False))
print("\nPart 3 - snp500.txt:")
print(f"  m={m}, n={n}, v_i=prices, p_j=1/{n}, x_ij=scenarios")
print("\nPart 4 - S&P 500:")
print(df_sp500.to_string(index=False))
print("\n" + "="*70)